## Final Project 
## Brainster DS x Parkinson's Disease Specifications

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import partial
import re
from scipy.stats import skew, kurtosis
from sklearn.model_selection import cross_validate, cross_val_score, train_test_split, KFold, StratifiedKFold

## Path to  User files

In [2]:
user_root = "D:/Brainster/Final Project/ArchivedUsers/ArchivedUsers/" # change to your path
user_fn_list = os.listdir(user_root)

### Function for read User files
#### cleaning the data

In [3]:
def read_one_file(fn, root):
    out = dict()
    with open(root + fn) as f:
        for line in f.readlines():
            k, v = line.split(": ")
            out[k] = v.strip()
            out['ID'] = re.findall(r'_(\w+)\.', fn)[0]
    return out

## Read all user files in list

In [4]:
users_list = list(map(partial(read_one_file, root=user_root), user_fn_list))

## Convert list to dataframe
## Cleaning the data

In [5]:
users = pd.DataFrame(users_list)
users.replace('------', np.nan, inplace=True)
users.replace('', np.nan, inplace=True)
users['Levadopa'] = users['Levadopa'] == 'True'
users['MAOB'] = users['MAOB'] == 'True'
users['Parkinsons'] = users['Parkinsons'] == 'True'
users['Tremors'] = users['Tremors'] == 'True'
users['Other'] = users['Other'] == 'True'

In [6]:
users

,BirthYear,ID,Gender,Parkinsons,Tremors,DiagnosisYear,Sided,UPDRS,Impact,Levadopa,DA,MAOB,Other
0,1959,0QAZFRHQHW,Female,False,False,NaN,None,Don't know,NaN,False,False,False,False
1,1944,1HOEBIGASW,Male,False,False,NaN,None,Don't know,NaN,False,False,False,False
2,1936,1XNJCXS3EY,Male,False,False,NaN,None,Don't know,NaN,False,False,False,False
3,1936,3DIXPRIOSW,Male,False,False,NaN,None,Don't know,NaN,False,False,False,False
4,1950,48DZPAJ5NS,Male,True,False,2010,None,Don't know,Mild,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,1948,YQSGN9BMVK,Male,False,False,NaN,None,Don't know,NaN,False,False,False,False
109,1953,YWMIQIQND3,Female,True,True,2016,Right,Don't know,Mild,False,False,False,False
110,1928,YYPKGX6B24,Male,False,False,NaN,None,Don't know,NaN,False,False,False,False
111,1947,Z2UPVHHGBE,Female,True,True,2015,Right,Don't know,Mild,False,False,False,True


## Function for read Tappy files

#### cleaning the data and fix corupted data

In [7]:
keys_root = "D:/Brainster/Final Project/TappyData/TappyData/" # change to your path
keys_fn_list = os.listdir(keys_root)

In [8]:
def read_one_key_file(fn, root):
    # try:
    df = pd.read_csv(root + fn, delimiter='\t', header=None, on_bad_lines='skip',
                         usecols=range(8), low_memory=False)
    df.columns = ['ID', 'Date', 'TS', 'Hand', 'HoldTime', 'Direction', 'LatencyTime', 'FlightTime']
    df = df[df['ID'].apply(lambda x: len(str(x)) == 10)
                   & df['Date'].apply(lambda x: len(str(x)) == 6)
                   & df['TS'].apply(lambda x: len(str(x)) == 12)
                   & np.in1d(df['Hand'], ["L", "R", "S"])
                   & df['HoldTime'].apply(lambda x: re.search(r"[^\d.]", str(x)) is None)
                   & np.in1d(df['Direction'], ['LL', 'LR', 'RL', 'RR', 'LS', 'SL', 'RS', 'SR', 'RR'])
                   & df['LatencyTime'].apply(lambda x: re.search(r"[^\d.]", str(x)) is None)
                   & df['FlightTime'].apply(lambda x: re.search(r"[^\d.]", str(x)) is None)]
    df['HoldTime'] = df['HoldTime'].astype(float)
    df['LatencyTime'] = df['LatencyTime'].astype(float)
    df['FlightTime'] = df['FlightTime'].astype(float)
    return df

### Read all Tappy files in list

In [9]:
keys_list = list(map(partial(read_one_key_file, root=keys_root), keys_fn_list))

### Combine all Tappy files

In [10]:
keys = pd.concat(keys_list, ignore_index=True, axis=0)

In [11]:
keys.shape

(4675453, 8)

In [12]:
keys['ID'].nunique()

113

### Find users with sufficient data

In [13]:
user_w_sufficient_data = set((keys.groupby('ID').size() >= 2000).index)
user_eligible = set(users[((users['Parkinsons']) & (users['Impact'] == 'Mild') 
                       | (~users['Parkinsons']))
                      & (~users['Levadopa'])]['ID'])
user_valid = user_w_sufficient_data.intersection(user_eligible)

In [14]:
# len(user_valid)

In [15]:
valid_keys = keys[(keys['HoldTime'] > 0)
                   & (keys['LatencyTime'] > 0)
                   & (keys['HoldTime'] < 2000)
                   & (keys['LatencyTime'] < 2000)
                   & np.in1d(keys['ID'], list(user_valid))]

In [16]:
hold_by_user =  valid_keys[valid_keys['Hand'] != 'S'].groupby(['ID', 'Hand'])['HoldTime'].agg(['mean', 'std', skew, kurtosis])

In [17]:
hold_by_user

mean        std      skew    kurtosis
ID         Hand                                             
0QAZFRHQHW L      98.931818  23.869914  0.236277    0.843128
           R     101.595749  37.219557  7.474979  109.926842
1HOEBIGASW L      66.280645  11.988617 -0.026245   -0.385415
           R      65.036667  11.677166  0.252502   -0.199870
1XNJCXS3EY L     153.702407  53.213749  0.264109    0.605739
...                     ...        ...       ...         ...
YWMIQIQND3 R     143.413333  45.440996 -0.334810    2.471268
YYPKGX6B24 L     148.072662  36.175880  0.668726    1.086600
           R     143.832754  36.258379  2.413969   18.717849
Z2UPVHHGBE L     131.337228  64.860916 -0.260596   -0.180302
           R     122.645700  63.894738  0.424304    1.121908

[162 rows x 4 columns]

In [18]:
latency_by_user = valid_keys[np.in1d(valid_keys['Direction'], ['LL', 'LR', 'RL', 'RR'])].groupby(['ID', 'Direction'])['LatencyTime'].agg(['mean', 'std', skew, kurtosis])

In [19]:
latency_by_user

mean         std      skew  kurtosis
ID         Direction                                            
0QAZFRHQHW LL         406.716242  162.606085  0.375895 -0.521971
           LR         411.718182  196.200749  0.247982 -1.102464
           RL         430.258974  178.154385  0.282171 -0.923822
           RR         365.736471  155.345904  0.572132 -0.265011
1HOEBIGASW LL         390.058824  199.162594  0.360494 -1.210490
...                          ...         ...       ...       ...
YYPKGX6B24 RR         545.699209  139.492600 -0.127751 -0.835330
Z2UPVHHGBE LL         240.875223  108.920795  1.617564  3.900310
           LR         259.178655  158.669496  1.363851  1.344283
           RL         197.612920  102.872797  2.389999  7.714524
           RR         275.028715  148.503021  1.110144  1.098808

[324 rows x 4 columns]

In [20]:
hold_by_user_flat = hold_by_user.unstack()
hold_by_user_flat.columns = ['_'.join(col).strip() for col in hold_by_user_flat.columns.values]
hold_by_user_flat['mean_hold_diff'] = hold_by_user_flat['mean_L'] - hold_by_user_flat['mean_R']
hold_by_user_flat.head()

,mean_L,mean_R,std_L,std_R,skew_L,skew_R,kurtosis_L,kurtosis_R,mean_hold_diff
ID,,,,,,,,,
0QAZFRHQHW,98.931818,101.595749,23.869914,37.219557,0.236277,7.474979,0.843128,109.926842,-2.663930
1HOEBIGASW,66.280645,65.036667,11.988617,11.677166,-0.026245,0.252502,-0.385415,-0.199870,1.243978
1XNJCXS3EY,153.702407,105.622423,53.213749,27.036622,0.264109,2.790381,0.605739,23.913747,48.079984
3DIXPRIOSW,147.626087,167.039039,47.259923,56.045862,0.665327,0.260479,0.592999,0.476607,-19.412952
48DZPAJ5NS,125.182493,126.045471,21.090258,20.136466,0.585148,0.308470,4.632557,5.007985,-0.862979


In [21]:
latency_by_user_flat = latency_by_user.unstack()
latency_by_user_flat.columns = ['_'.join(col).strip() for col in latency_by_user_flat.columns.values]
latency_by_user_flat['mean_LR_RL_diff'] = latency_by_user_flat['mean_LR'] - latency_by_user_flat['mean_RL']
latency_by_user_flat['mean_LL_RR_diff'] = latency_by_user_flat['mean_LL'] - latency_by_user_flat['mean_RR']
latency_by_user_flat.head()

,mean_LL,mean_LR,mean_RL,mean_RR,std_LL,std_LR,std_RL,std_RR,skew_LL,skew_LR,skew_RL,skew_RR,kurtosis_LL,kurtosis_LR,kurtosis_RL,kurtosis_RR,mean_LR_RL_diff,mean_LL_RR_diff
ID,,,,,,,,,,,,,,,,,,
0QAZFRHQHW,406.716242,411.718182,430.258974,365.736471,162.606085,196.200749,178.154385,155.345904,0.375895,0.247982,0.282171,0.572132,-0.521971,-1.102464,-0.923822,-0.265011,-18.540793,40.979771
1HOEBIGASW,390.058824,600.433333,536.407143,394.647059,199.162594,185.713598,78.443370,220.782942,0.360494,-0.815568,0.604678,0.250220,-1.210490,-0.446691,-0.267530,-1.227361,64.026190,-4.588235
1XNJCXS3EY,347.882547,313.541489,310.799454,322.170833,101.977747,97.680669,98.038127,82.901327,-1.332784,0.008248,0.037478,-0.284840,1.946994,-0.715397,-0.723288,0.725222,2.742036,25.711714
3DIXPRIOSW,528.670445,575.478761,501.274093,493.779630,137.542548,148.448891,143.404457,163.597901,-0.192664,-0.259537,0.357609,-0.368179,-0.096967,-1.163198,-0.854153,0.201365,74.204668,34.890816
48DZPAJ5NS,300.323155,335.508287,321.131506,332.621036,81.749015,81.219856,88.467507,90.876395,0.361974,0.029114,0.143947,-0.048925,-0.564402,-0.719146,-0.929742,-1.048276,14.376781,-32.297880


In [22]:
combined = pd.concat([hold_by_user_flat, latency_by_user_flat], axis=1)

In [23]:
full_set = pd.merge(combined.reset_index(), users[['ID', 'Parkinsons']], on='ID')
full_set.set_index('ID', inplace=True)
full_set.dropna(inplace=True)  # should investigate why there are NAs despite choosing sequence length >= 2000
full_set

,mean_L,mean_R,std_L,std_R,skew_L,skew_R,kurtosis_L,kurtosis_R,mean_hold_diff,mean_LL,...,skew_LR,skew_RL,skew_RR,kurtosis_LL,kurtosis_LR,kurtosis_RL,kurtosis_RR,mean_LR_RL_diff,mean_LL_RR_diff,Parkinsons
ID,,,,,,,,,,,,,,,,,,,,,
0QAZFRHQHW,98.931818,101.595749,23.869914,37.219557,0.236277,7.474979,0.843128,109.926842,-2.663930,406.716242,...,0.247982,0.282171,0.572132,-0.521971,-1.102464,-0.923822,-0.265011,-18.540793,40.979771,False
1HOEBIGASW,66.280645,65.036667,11.988617,11.677166,-0.026245,0.252502,-0.385415,-0.199870,1.243978,390.058824,...,-0.815568,0.604678,0.250220,-1.210490,-0.446691,-0.267530,-1.227361,64.026190,-4.588235,False
1XNJCXS3EY,153.702407,105.622423,53.213749,27.036622,0.264109,2.790381,0.605739,23.913747,48.079984,347.882547,...,0.008248,0.037478,-0.284840,1.946994,-0.715397,-0.723288,0.725222,2.742036,25.711714,False
3DIXPRIOSW,147.626087,167.039039,47.259923,56.045862,0.665327,0.260479,0.592999,0.476607,-19.412952,528.670445,...,-0.259537,0.357609,-0.368179,-0.096967,-1.163198,-0.854153,0.201365,74.204668,34.890816,False
48DZPAJ5NS,125.182493,126.045471,21.090258,20.136466,0.585148,0.308470,4.632557,5.007985,-0.862979,300.323155,...,0.029114,0.143947,-0.048925,-0.564402,-0.719146,-0.929742,-1.048276,14.376781,-32.297880,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YIA9DW5AGQ,74.926898,73.006689,16.091520,20.954124,2.001092,8.317765,10.657469,110.893628,1.920209,233.183499,...,0.955728,0.593330,1.017654,2.243062,0.729124,-0.463778,3.072539,-33.240364,-29.417223,False
YQSGN9BMVK,101.932172,114.030694,43.697289,38.213710,4.383692,0.123665,151.434302,2.784825,-12.098522,284.901879,...,1.784531,1.297874,1.679683,1.255646,3.511398,1.472475,3.726144,-40.103793,13.453680,False
YWMIQIQND3,103.910159,143.413333,22.957967,45.440996,0.387375,-0.334810,4.793937,2.471268,-39.503175,249.626144,...,0.277019,0.734572,-0.697133,1.262639,-0.990434,0.379500,0.050765,15.101885,-100.736933,True


In [24]:
full_set['Parkinsons'] = full_set['Parkinsons'].astype(int)


In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(full_set.drop(columns=['Parkinsons']), full_set['Parkinsons'], test_size=0.2, random_state=42)

In [27]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_depth=5)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5)

In [28]:
rf.score(X_test, y_test)

0.5882352941176471

In [29]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [30]:
lda_model = LDA()

In [31]:
lda_model.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [32]:
rf = RandomForestClassifier(n_estimators=100, max_depth=5)
rf.fit(lda_model.transform(X_train), y_train)

RandomForestClassifier(max_depth=5)

In [33]:
rf.score(lda_model.transform(X_test), y_test)

0.8235294117647058

In [34]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error,roc_auc_score

In [35]:
y_pred = rf.predict(lda_model.transform(X_test))

In [36]:

mse = mean_squared_error(y_test, y_pred) # sum over (y_test - y_pred) ^ 2
mae = mean_absolute_error(y_test, y_pred) # sum over abs(y_test - y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean squared error", mse)
print("Mean absolute error", mae)
print("R2 score", r2)

Mean squared error 0.17647058823529413
Mean absolute error 0.17647058823529413
R2 score 0.29166666666666663


In [37]:
auc = roc_auc_score(y_test, y_pred)
auc

0.8194444444444443

In [38]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV 

In [39]:
param_grid = { 
    'n_estimators': [25, 50, 100, 150], 
    'max_features': ['sqrt', 'log2', None], 
    'max_depth': [3, 6, 9], 
    'max_leaf_nodes': [3, 6, 9], 
} 

In [40]:
grid_search = GridSearchCV(RandomForestClassifier(), 
                           param_grid=param_grid) 
grid_search.fit(lda_model.transform(X_train), y_train) 
print(grid_search.best_estimator_) 

RandomForestClassifier(max_depth=3, max_leaf_nodes=9)


In [41]:
best_rf = RandomForestClassifier(n_estimators=50, 
                                 max_depth=9,
                                 max_features=None,
                                 max_leaf_nodes=3,
                                 random_state=42)

In [42]:
best_rf.fit(lda_model.transform(X_train), y_train)

RandomForestClassifier(max_depth=9, max_features=None, max_leaf_nodes=3,
                       n_estimators=50, random_state=42)

In [43]:
best_rf.score(lda_model.transform(X_test), y_test)

0.7647058823529411

In [44]:
pred_rf = best_rf.predict(lda_model.transform(X_test))

In [45]:
auc = roc_auc_score(y_test, pred_rf)
auc

0.763888888888889

In [46]:
from xgboost import XGBClassifier

In [47]:
xgb = XGBClassifier(random_state=42)

In [48]:
xgb.fit(lda_model.transform(X_train), y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [49]:
xgb.score(lda_model.transform(X_test), y_test)

0.8235294117647058

In [50]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# Define the hyperparameter space
space = {
    'max_depth': hp.choice('max_depth', np.arange(1, 14, dtype=int)),
    'learning_rate': hp.loguniform('learning_rate', -5, -2),
    'subsample': hp.uniform('subsample', 0.5, 1)
}

# Define the objective function to minimize
def objective(params):
    xgb_model = xgb.XGBClassifier(**params)
    xgb_model.fit(lda_model.transform(X_train), y_train)
    y_pred = xgb_model.predict(lda_model.transform(X_test))
    score = roc_auc_score(y_test, y_pred)
    return {'loss': -score, 'status': STATUS_OK}

# Perform the optimization
best_params = fmin(objective, space, algo=tpe.suggest, max_evals=100)
print("Best set of hyperparameters: ", best_params)

100%|██████████| 100/100 [00:01<00:00, 64.68trial/s, best loss: -0.8263888888888888]
Best set of hyperparameters:  {'learning_rate': 0.05195305967050769, 'max_depth': 11, 'subsample': 0.7719878230888924}


In [51]:
model = xgb.XGBClassifier(**best_params)

In [52]:
model.fit(lda_model.transform(X_train), y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05195305967050769,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=11, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [53]:
model.score(lda_model.transform(X_test), y_test)

0.8235294117647058

In [54]:
pred = model.predict(lda_model.transform(X_test))

In [55]:
auc = roc_auc_score(y_test, pred)
auc

0.8263888888888888

In [86]:
from catboost import CatBoostClassifier

In [114]:
model_cat = CatBoostClassifier(learning_rate=0.03,iterations=1000,depth=8,loss_function='Logloss',min_data_in_leaf=10,random_seed=42)
model_cat.fit(lda_model.transform(X_train), y_train)    

0:	learn: 0.6698742	total: 1.05ms	remaining: 1.05s
1:	learn: 0.6513510	total: 1.84ms	remaining: 916ms
2:	learn: 0.6335083	total: 2.66ms	remaining: 883ms
3:	learn: 0.6173891	total: 3.51ms	remaining: 874ms
4:	learn: 0.6025065	total: 4.22ms	remaining: 839ms
5:	learn: 0.5910925	total: 4.91ms	remaining: 814ms
6:	learn: 0.5792643	total: 5.6ms	remaining: 795ms
7:	learn: 0.5655963	total: 6.33ms	remaining: 786ms
8:	learn: 0.5578955	total: 6.66ms	remaining: 733ms
9:	learn: 0.5454837	total: 7.36ms	remaining: 728ms
10:	learn: 0.5354536	total: 8.02ms	remaining: 721ms
11:	learn: 0.5226666	total: 8.96ms	remaining: 738ms
12:	learn: 0.5158548	total: 9.42ms	remaining: 715ms
13:	learn: 0.5075271	total: 10.1ms	remaining: 713ms
14:	learn: 0.5013901	total: 10.8ms	remaining: 710ms
15:	learn: 0.4922912	total: 11.6ms	remaining: 711ms
16:	learn: 0.4864855	total: 12ms	remaining: 693ms
17:	learn: 0.4775378	total: 12.7ms	remaining: 694ms
18:	learn: 0.4710281	total: 13.5ms	remaining: 696ms
19:	learn: 0.4636049	tota

In [115]:
pred = model_cat.predict_proba(lda_model.transform(X_test))
auc = roc_auc_score(y_test, pred[:, 1])
auc

0.9097222222222222